In [1]:
import numpy as np

In [6]:
def geotransform(coefficients_lst, pixel_coords):
    GT = coefficients_lst
    x_pixel, y_pixel = pixel_coords
    
    X_geo = GT[0] + x_pixel * GT[1] + y_pixel * GT[2]
    Y_geo = GT[3] + x_pixel * GT[4] + y_pixel * GT[5]
    return X_geo, Y_geo

In [7]:
coefficients = (98.47236, 0.00027777, 0, 3.6506945, 0, -0.0002777)

In [13]:
pixel_coords = [(4215, 3054), (7004, 8008), (12626, 8331), (11658, 2133)]

In [14]:
[geotransform(coefficients, (x,y)) for x,y in pixel_coords]

[(99.64316054999999, 2.8025987000000003),
 (100.41786108, 1.4268729000000002),
 (101.97948402, 1.3371758000000002),
 (101.71060265999999, 3.0583604)]

In [15]:
xp = np.array([4215, 7004, 12626, 11658])
yp = np.array([3054, 8008, 8331, 2133])
X = np.array([99.643, 100.418, 101.98, 101.711])
Y = np.array([2.802, 1.426, 1.336, 3.058])

In [16]:
xp_mean = xp.mean()
yp_mean = yp.mean()

e = xp - xp_mean
n = yp - yp_mean


In [17]:
A = np.column_stack([
    np.ones_like(e),  # constant term
    e,                # pixel offset
    n                 # line offset
])


In [18]:
E, *_ = np.linalg.lstsq(A, X, rcond=None)
N, *_ = np.linalg.lstsq(A, Y, rcond=None)


In [20]:
def pixel_to_geo(xp, yp):
    e = xp - xp_mean
    n = yp - yp_mean
    X = E[0] + E[1]*e + E[2]*n
    Y = N[0] + N[1]*e + N[2]*n
    return X, Y

In [21]:
for i in range(4):
    print(pixel_to_geo(xp[i], yp[i]), (X[i], Y[i]))


(np.float64(99.64301478490721), np.float64(2.8020989404075847)) (np.float64(99.643), np.float64(2.802))
(np.float64(100.417979867243), np.float64(1.425865271850943)) (np.float64(100.418), np.float64(1.426))
(np.float64(101.98001688658404), np.float64(1.3361130048016014)) (np.float64(101.98), np.float64(1.336))
(np.float64(101.71098846126576), np.float64(3.0579227829398707)) (np.float64(101.711), np.float64(3.058))


In [22]:
GT0 = E[0] - E[1]*xp_mean - E[2]*yp_mean
GT1 = E[1]
GT2 = E[2]
GT3 = N[0] - N[1]*xp_mean - N[2]*yp_mean
GT4 = N[1]
GT5 = N[2]

GT = [GT0, GT1, GT2, GT3, GT4, GT5]


In [23]:
GT

[np.float64(98.47186910073182),
 np.float64(0.00027784294468229895),
 np.float64(1.2335409136205253e-08),
 np.float64(3.650517944190537),
 np.float64(-4.053945286025745e-09),
 np.float64(-0.00027780023457877256)]

# Solving Equations Only